In [1]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

ps = PorterStemmer()
new_text = "It is important to by very pythonly while you are pythoning with python. All pythoners have pythoned poorly at least once."
words = word_tokenize(new_text)


In [1]:
def pre_train_data(train_dir):
    for root,dirs,files in os.walk(train_dir):
        for name in dirs:
            subdirpath=os.path.join(root,name)
            if name=='ham':
                ham_emails=[os.path.join(subdirpath,f) for f in os.listdir(subdirpath)]
                ham_size=len(os.listdir(subdirpath))
            elif name=='spam':
                spam_emails=[os.path.join(subdirpath,f) for f in os.listdir(subdirpath)]
                spam_size=len(os.listdir(subdirpath))

    emails=ham_emails+spam_emails
    train_size=ham_size+spam_size
    train_labels=np.zeros(train_size,dtype=np.int)
    train_labels[ham_size:]=1
    return emails,train_labels
    
        

In [2]:
def pre_test_data(test_dir):
    txtIDs=[]
    emails=[]
    for file in os.listdir(test_dir):
        txtID=file[11:-4]
        txtIDs.append(int(txtID))
        emails.append(os.path.join(test_dir,file))
    #print(txtIDs)
    #print(emails)
    sorted_emails = [x for _,x in sorted(zip(txtIDs,emails))]
    #print(sorted_emails)
    return sorted_emails

In [3]:
import os
import numpy as np
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

def make_Dictionary(train_emails):
    all_words=[]
    for email in train_emails:
#         with open(email,mode="r",encoding='ISO-8859-1') as m:
        with open(email,'rb') as m:  
            for i, line in enumerate(m):
                words=line.split()
                all_words+=words
    dictionary=Counter(all_words)
    list_to_remove=dictionary.keys()
    for item in list(list_to_remove):
        if item.isalpha()==False:
            del dictionary[item]
        elif len(item)==1:
            del dictionary[item]
    dictionary=dictionary.        
    dictionary = dictionary.most_common(3000)
    return dictionary

SyntaxError: invalid syntax (<ipython-input-3-0ed5557dc6d1>, line 21)

In [6]:
import os
import numpy as np
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize


ps = PorterStemmer()
def make_Dictionary2(train_emails):
    all_words=[]
    ps = PorterStemmer()
    for email in train_emails:
        ##with open(email,mode="r",encoding='ISO-8859-1') as m:
        with open(email,'rb') as m:          
            data=m.read().replace('\n', '')
#             data=m.read()
            words = word_tokenize(data)
            for w in words:
                w=ps.stem(w)
            all_words+=words
    dictionary=Counter(all_words)
    list_to_remove=dictionary.keys()
    for item in list(list_to_remove):
        if item.isalpha()==False:
            del dictionary[item]
        elif len(item)==1:
            del dictionary[item]
        
    dictionary = dictionary.most_common(3000)
    return dictionary

In [7]:
def extract_features(train_emails):
    features_matrix=np.zeros((len(train_emails),3000))
    txtID=0
    for fil in train_emails:
        #with open(fil, mode="r",encoding='ISO-8859-1') as fi:
        with open(fil,'rb') as fi:
            all_words=[]
            data=fi.read().replace('\n', '')
            words = word_tokenize(data)
            for w in words:
                w=ps.stem(w)
            all_words+=words
#             all_words=[]
#             words = word_tokenize(m)
#             all_words+=words
#             for line in fi:
#                 words=line.split()
#                 all_words+=words
        
            for i,d in enumerate(dictionary):
                if d[0] in all_words:
                    wordID=i
                    features_matrix[txtID,wordID]=all_words.count(d[0])
            txtID=txtID+1
    return features_matrix

In [8]:
#from io import open
import os 
import numpy as np
from collections import Counter
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.svm import SVC, NuSVC, LinearSVC
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import csv
import sys  

reload(sys)  
sys.setdefaultencoding('ISO-8859-1')

#train_dir='/Users/caixiuhong/Dropbox/Caiprivate/machine learning/email_classification_data/train_data'
train_dir='./hw3data/train_data'
train_emails,train_labels=pre_train_data(train_dir)
test_dir='./hw3data/test_data'
test_emails = pre_test_data(test_dir)


dictionary=make_Dictionary2(train_emails)
train_matrix=extract_features(train_emails)
test_matrix=extract_features(test_emails)
print('finish')

In [13]:
model2=MultinomialNB()
# model2=SVC(kernel='linear', gamma=2)
# model1.fit(train_matrix, train_labels)
model2.fit(train_matrix,train_labels)



#test_dir='/Users/caixiuhong/Dropbox/Caiprivate/machine learning/email_classification_data/test_data'

#test_labels=np.zeros(len(test_emails))
#test_labels[130:360]=1
#print(sum(test_matrix==1))
# result1=model1.predict(test_matrix)
result2=model2.predict(test_matrix)

# for email_id, labels in enumerate(result1):
#     print(email_id+1, labels)

In [14]:
out = open('result2.csv', "wb")
out.write("email_id,labels\n")

rows = ['']*result2.shape[0] # predefine or use append
for num in range(0, result2.shape[0]):
    label = 0 #TODO; classify here
    rows[num] = "%d,%d\n"%(num+1,result2[num])

out.writelines(rows)
out.close()